In [27]:
import numpy as np
import matplotlib.pyplot as plt
import random
import tensorflow as tf  
import pathlib
import os
import re
import cv2
from tensorflow.keras.models import Sequential

ModuleNotFoundError: No module named 'cv2'

<h3>Configure Tensorflow</h3>

In [3]:
config = tf.compat.v1.ConfigProto()  
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU  
config.log_device_placement = True  # to log device placement (on which device the operation ran)  
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.compat.v1.Session(config=config)  
tf.compat.v1.keras.backend.set_session(sess)  # set this TensorFlow session as the default session for Keras  


Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080, pci bus id: 0000:08:00.0, compute capability: 7.5



<h3> Load dataset </h3>

In [26]:
dataset_path = './histology_slides/'
dataset_path = pathlib.Path(dataset_path)

labels = [] # 0: benign, 1: malignant

num_malign = 0

for p in dataset_path.rglob("*"):
    filename, file_extension = os.path.splitext(p)
    
    if file_extension == '.png':
        #<BIOPSY_PROCEDURE>_<TUMOR_CLASS>_<TUMOR_TYPE>_<YEAR>-<SLIDE_ID>-<MAGNIFICATION>-<SEQ>
        meta = re.split('_|-', os.path.basename(filename))
        if meta[1] == 'M':
            labels.append(1)
            num_malign += 1
        else:
            labels.append(0)
        
print('malignant percentage: ' + str(num_malign / len(labels)))

malignant percentage: 0.6864331773928436


In [24]:
labels[6000]


1